# Dressmaker - Hard
You may need to create views to complete these questions - but you do not have permission to create tables or views in the default schema. Your SQL commands are executed by user scott in schema gisq - you may create or drop views and tables in schema scott but not in gisq.

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app17-3")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@19b92abc

In [2]:
import spark.implicits._
import spark.sqlContext.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

import spark.sqlContext.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@249010a7

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val jmcust = hiveCxt.table("sqlzoo.jmcust")
val dressmaker = hiveCxt.table("sqlzoo.dressmaker")
val dress_order = hiveCxt.table("sqlzoo.dress_order")
val construction = hiveCxt.table("sqlzoo.construction")
val quantities = hiveCxt.table("sqlzoo.quantities")
val order_line = hiveCxt.table("sqlzoo.order_line")
val garment = hiveCxt.table("sqlzoo.garment")
val material = hiveCxt.table("sqlzoo.material")

jmcust: DataFrame = [c_no: int, c_name: string ... 2 more fields]
dressmaker: DataFrame = [d_no: int, d_name: string ... 2 more fields]
dress_order: DataFrame = [order_no: int, cust_no: int ... 2 more fields]
construction: DataFrame = [maker: int, order_ref: int ... 3 more fields]
quantities: DataFrame = [style_q: int, size_q: int ... 1 more field]
order_line: DataFrame = [order_ref: int, line_no: int ... 3 more fields]
garment: DataFrame = [style_no: int, description: string ... 2 more fields]
material: DataFrame = [material_no: int, fabric: string ... 3 more fields]

## 1.
When creating a view in scott you must specify the schema name of the sources and the destination.

## 2.
It is decided to review the materials stock. How much did each material contribute to turnover in 2002?

In [5]:
(material
 .join(order_line, (material("material_no") === order_line("ol_material")))
 .join(quantities, ((order_line("ol_size") === quantities("size_q")) &&
                       (order_line("ol_style") === quantities("style_q"))))
 .join(dress_order.filter(year(dress_order("order_date")) === 2002),
       (order_line("order_ref") === dress_order("order_no")))
 .withColumn("cost", col("cost") * col("quantity"))
 .groupBy("material_no", "fabric", "colour", "pattern")
 .agg(sum("quantity"), sum("cost"))
 .withColumn("sum(cost)", round($"sum(cost)", 2))
 .showHTML())

material_no,fabric,colour,pattern,sum(quantity),sum(cost)
7,Polyester,Pale Yellow,Printed,4.3,10.96
10,Silk,Green Abstract,Printed,8.3,124.5
5,Cotton,Black Dotted,Woven,6.4,19.2
1,Silk,Black,Plain,4.9,34.3
14,Cotton,Green Abstract,Printed,4.5,15.75
11,Rayon,Red/Orange,Printed,9.7,38.8
6,Cotton,Red Stripe,Woven,2.2,6.6
8,Cotton,Blue Stripe,Woven,4.2,12.6
12,Serge,Navy Blue,Woven,4.5,24.75
4,Cotton,Green Stripe,Woven,2.2,6.6


## 3.
An order for shorts has just been placed and the work is to be distributed amongst the workforce, and we wish to know how busy the shorts makers are. For each of the workers who have experience of making shorts show the number of hours work that she is currently committed to, assuming a meagre wage of £4.50 per hour

In [6]:
(dressmaker
 .join(dressmaker
        .join(construction, (dressmaker("d_no") === construction("maker")))
        .join(dress_order, (construction("order_ref") === dress_order("order_no")))
        .join(order_line, (dress_order("order_no") === order_line("order_ref")))
        .join(garment.filter(lower(trim(garment("description"))) === "shorts"), 
              (order_line("ol_style") === garment("style_no")))
        .select("d_no")
        .distinct().alias("shorts"),
       "d_no")
 .join(construction, (dressmaker("d_no") === construction("maker")))
 .join(dress_order.filter(dress_order("completed") === "N"), 
       (col("order_ref") === dress_order("order_no")))
 .join(order_line, (col("order_no") === order_line("order_ref")))
 .join(garment.withColumn("hrs", garment("labour_cost")/4.5),  
       (col("ol_style") === garment("style_no")))
 .join(quantities, ((col("ol_size") === quantities("size_q")) &&
                    (col("ol_style") === quantities("style_q"))))
 .groupBy("d_name")
 .sum("hrs")
 .withColumn("sum(hrs)", round($"sum(hrs)", 2))
 .showHTML())

d_name,sum(hrs)
Mr Seam,28.17
Ms Sew,18.61
Miss Stitch,49.17
Miss Pins,28.17
Mr Needles,18.61
Mr Taylor,18.61


## 4.
"Big spender of the year" is the customer who spends the most on high value items. Identify the "Big spender of the year 2002" if the "high value" threshold is set at £30. Also who would it be if the threshold was £20 or £50?

In [7]:
val t = (order_line
     .join(quantities, ((order_line("ol_style") === quantities("style_q")) && 
                        (order_line("ol_size") === quantities("size_q"))))
     .join(garment, (order_line("ol_style") === garment("style_no")))
     .join(material, (order_line("ol_material") === material("material_no")))
     .withColumn("tot_cost", col("labour_cost") + col("quantity") * col("cost")))

val c = (dress_order.filter(year(dress_order("order_date")) === 2002)
     .join(jmcust, (dress_order("cust_no") === jmcust("c_no")))
     .join(t.select("order_ref", "line_no", "tot_cost"), 
           (col("order_no") === t("order_ref"))))

val lst = List(20, 30, 50).map(thres => (
    c.filter($"tot_cost" >= thres)
     .groupBy("c_name")
     .agg(sum("tot_cost").alias("tot_cost"))
     .orderBy(col("tot_cost").desc)
     .limit(1)
     .select("c_name", "tot_cost")
     .withColumn("tot_cost", round($"tot_cost", 2))
     .withColumn("thres", lit(thres))))

lst.reduce(_.union(_)).showHTML()

c_name,tot_cost,thres
Mr Brass,198.54,20
Ms White,173.55,30
Mr Brass,72.0,50


t: DataFrame = [order_ref: int, line_no: int ... 16 more fields]
c: DataFrame = [order_no: int, cust_no: int ... 9 more fields]
lst: List[DataFrame] = List(
  [c_name: string, tot_cost: double ... 1 more field],
  [c_name: string, tot_cost: double ... 1 more field],
  [c_name: string, tot_cost: double ... 1 more field]
)

## 5.
Who is the fastest at making trousers?

In [8]:
(dressmaker.join(construction, (dressmaker("d_no") === construction("maker")))
 .join(order_line, ((construction("order_ref") === order_line("order_ref")) && 
                    (construction("line_ref") === order_line("line_no"))))
 .join(garment.filter(lower(trim(garment("description"))) === "trousers"), 
       (order_line("ol_style") === garment("style_no")))
 .withColumn("days", (to_date(col("finish_date"))-to_date(col("start_date"))).cast("integer"))
 .select("d_no", "d_name", "days")
 .na.drop(Array("days"))
 .orderBy("days")
 .limit(1)
 .showHTML())

d_no,d_name,days
3,Mr Needles,3


## 6.
"Employee of the month" is the seamstress who completes the greatest value of clothes. Show the "employees of the month" for months in 2002.

In [9]:
(material
 .join(order_line, (material("material_no") === order_line("ol_material")))
 .join(quantities, ((order_line("ol_style") === quantities("style_q")) && 
                    (order_line("ol_size") === quantities("size_q"))))
 .join(garment, (order_line("ol_style") === garment("style_no")))
 .join(construction.filter(year(to_date(construction("start_date"))) === 2002), 
       ((construction("order_ref") === order_line("order_ref")) && 
           (construction("line_ref") === order_line("line_no"))))
 .join(dressmaker, (construction("maker") === dressmaker("d_no")))
 .withColumn("month", month(to_date(col("start_date"))))
 .withColumn("val", col("quantity") * col("cost") + col("labour_cost"))
 .groupBy("d_name", "month")
 .agg(sum("val").alias("val"))
 .withColumn("sn", rank().over(
     Window.partitionBy("month").orderBy(col("val").desc)))
 .filter(col("sn") === 1)
 .select("d_name", "month", "val")
 .showHTML())

d_name,month,val
Miss Stitch,1,49.0
Mrs Hem,2,122.25
Miss Stitch,3,97.2


In [10]:
spark.stop()